# Notebook to build the CV-Setup on which to improve on

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from glob import glob
import os
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import timm
from torchinfo import summary
from copy import deepcopy
import segmentation_models_pytorch as smp
from torchinfo import summary

In [ ]:
!pwd

In [ ]:
folder = '/home/olli/Projects/Kaggle/Vesuvius'

# There are 3 scans so a 3-fold-Cross-Validation makes sense

## For this first create each fold (each fold is one scan)

In [ ]:
fold_1_path = os.path.join(folder, 'train/1')
fold_2_path = os.path.join(folder, 'train/2')
fold_3_path = os.path.join(folder, 'train/3')

In [ ]:
scans_1 = glob(fold_1_path + '/surface_volume/*.tif')
scans_2 = glob(fold_2_path + '/surface_volume/*.tif')
scans_3 = glob(fold_3_path + '/surface_volume/*.tif')

In [ ]:
scans_1.sort()
scans_2.sort()
scans_3.sort()

In [ ]:
for i in [scans_1, scans_2, scans_3]:
    print(i[0], '\n', i[-1], '\n')

In [ ]:
folds = [scans_1, scans_2, scans_3]

In [ ]:
fold_paths = [fold_1_path, fold_2_path, fold_3_path]

## Now create a generic dataset for the folds

In [ ]:
'''This Dataset-class takes as input the folder of a scan as well as all paths to the volumes (sorted).
   Each scan is returned as a 3D-volume as well as the label.
   The volume is normalized, however this can not be in one part.
   The reason is a limit in RAM: One scan takes ~16GB of RAM and normalizing the full volume crashed the kernel.
    -> Needs more then 33.6GB of available RAM.
   Thats why the normalization is done in slices: Each scan layer will be normalized to a fix mean & std.
    -> This doesn't take much additional RAM.
   The depth means how many layers to load (max: 65)
'''

class Data(Dataset):
    def __init__(self, path, scans_paths, depth=10):
        
        self.scans = scans_paths
        self.paths = path
        self.depth = depth
        assert self.depth > 1 and self.depth <= 65
        
    def __len__(self):  # number of scans for this dataset
        return len(self.paths)
    
    def __getitem__(self, index):
        
        # load the label for the scan
        label_path = os.path.join(self.paths[index], 'inklabels.png')
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)  # no colors
        
        # create an empty volume and assign each scan layer; get the shape from the label (same as scan for h x w)
        height, width = label.shape
        scan = torch.empty(65, height, width)
        
        # now assign each layer at the correct dim
        for i in range(self.depth):  # 0...64
            scan_array = cv2.imread(folds[index][i], cv2.IMREAD_GRAYSCALE)
            scan[i, :, :] = torch.tensor(scan_array)
            scan[i, :, :] = scan[i, :, :].type(torch.float32)
            scan[i, :, :] = scan[i, :, :] / 255.0
            
            #normalize each slize on its own to a mean of 0.5 and a std of 0.5!
            scan[i, :, :] = (scan[i, :, :] - scan[i, :, :].mean()) / (scan[i, :, :].std() * 2.) + 0.5
            
            
        return scan, label

In [ ]:
'''Write a function that returns the different train/valid-datasets and -paths.
   Each fold once is the validation data, the rest is for training.'''


def create_train_valid():

    train_folds = ['tr_1', 'tr_2', 'tr_3']
    valid_folds = ['val_1', 'val_2', 'val_3']
    train_paths = ['tr_path_1', 'tr_path_2', 'tr_path_3']
    valid_paths = ['val_path_1', 'val_path_2', 'val_path_3']

    # assign each train/valid dataset the correct folds with i
    for i, train, valid in zip(range(3), train_folds, valid_folds):

        paths = deepcopy(fold_paths)
        flds = deepcopy(folds)

        globals()[valid_paths[i]] = [paths.pop(i)]  # i'ths index is valid rest is for train
        globals()[train_paths[i]] = paths

        globals()[valid_folds[i]] = [flds.pop(i)]
        globals()[train_folds[i]] = flds
        
    train_folds = [tr_1, tr_2, tr_3]
    valid_folds = [val_1, val_2, val_3]
    
    train_paths = [tr_path_1, tr_path_2, tr_path_3]
    valid_paths = [val_path_1, val_path_2, val_path_3]
    
    return train_folds, valid_folds, train_paths, valid_paths

In [ ]:
train_folds, valid_folds, train_paths, valid_paths = create_train_valid()

### Check if it worked to see each train-/valid-path

In [ ]:
for i, tr, val in zip(range(3), train_paths, valid_paths):
    print(f'Dataset {i}: Train-Path: {tr}; Valid_Path: {val}')

### Each valid fold is excluded for train so it worked

In [ ]:
for i in range(3):
    train_path = train_paths[i]
    valid_path = valid_paths[i]
    
    train_data = train_folds[i]
    valid_data = valid_folds[i]
    
    tr_ds = Data(path=train_path, scans_paths=train_data)
    val_ds = Data(path=valid_path, scans_paths=valid_data)
    
    print(f'Dataset {i + 1}: Train-samples: {tr_ds.__len__()}; Valid-samples: {val_ds.__len__()}')

# Now build a simple 2D Conv Net that predicts each layer 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
model = smp.Unet(
    encoder_name='se_resnext50_32x4d',        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights=None,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)

In [ ]:
summary(model, input_data=torch.randn(1, 1, 6336, 8192).type(torch.float32))